# Recommender Systems Deep-Dive Lab

There are many different approaches that we can take when creating recommender systems. In the Intro to Recommender Systems lesson and lab, we put together a user similarity based recommender that first calculated the similarities between users and then leveraged a rank-based item recommender within each group of similar customers. In other words, for a given user, our recommender found the top 5 customers who were the most similar to them, aggregated and ranked the purchases of those 5 customers, and then recommended the top 5 most popular products among that group of similar users to the customer.

In this lab, we are going to start out with the same data set, but we are going to dive deeper into the analysis of customers and products and look at an alternative way to generate recommendations.

We will begin by importing everything we will need for this lab (libraries, data set, etc.).

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform 

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20
...,...,...,...,...,...,...,...
68579,94547,Gerald,Donovan,687884,211,Wine - Prosecco Valdobiaddene,24
68580,57654,Lashonda,Roy,4373415,211,Wine - Prosecco Valdobiaddene,15
68581,36751,Glen,Frederick,2405597,211,Wine - Prosecco Valdobiaddene,10
68582,43270,Devon,Mc Daniel,4328852,211,Wine - Prosecco Valdobiaddene,11


## Data Preparation

We will then put together the foundational transformations of the data that we will need to eventually produce recommendations. The steps in this section should be familiar to you, as you would have had to tranform the data in this manner to create the user similarity based recommender in the Intro to Recommender Systems lab.

First, we will create a data frame that contains the total quantity of each product purchased by each customer.

In [4]:
customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()

Then, we want to create a matrix that has customers on one axis, products on the other, and the quantity purchased as the values. There will be many instances where a customer has not purchased a product, which by default will be expressed with a null value. We will want to replace those nulls with zeros by appending `.fillna(0)` to our pivot table.

In [5]:
prod_cust_pivot = customer_products.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)

prod_cust_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
prod_cust_pivot2 = prod_cust_pivot.copy()

In the pivot table we created, the rows represent the products and the columns represent the customers. Depending on what need to do with the matrix, we may instead need to transpose it so that the rows represent customers and the columns to represent products. We can do this easily by appending `.T` to our product customer matrix.

In [7]:
cust_prod_pivot = prod_cust_pivot.T
cust_prod_pivot.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Another thing we may want to do is normalize the values across rows or columns of the matrix so that all the values are between 0 and 1. Doing this for customers would help us identify customers that may have purcahsed a similar mix of products even though some of those customers may have purchased large quantities while other may have purchased smaller quantities. Doing this for products would help us better identify products that have been purchased by similar groups of customers regardless of the quantities purchased.

We can normalize across rows for each matrix as follows.

In [8]:
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)

## User Similarity Based Recommendations

The next step in creating recommendations is calculating similarities. For our user similarity based recommender, we calculated them between customers.

In [9]:
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

cust_dist.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.854082,0.859145,0.852783,0.856118,0.853239,0.849549,0.848801,0.851224,0.849892,...,0.855130,0.846812,0.850301,0.841254,0.852221,0.846485,0.852848,0.847637,0.856465,0.851614
200,0.854082,1.000000,0.857935,0.855900,0.856445,0.853129,0.863562,0.850521,0.857381,0.856621,...,0.864397,0.856014,0.862447,0.854629,0.856670,0.854046,0.868990,0.857312,0.861508,0.857184
264,0.859145,0.857935,1.000000,0.851801,0.860050,0.851262,0.855847,0.846677,0.854206,0.848997,...,0.861268,0.850766,0.854178,0.848507,0.858909,0.860153,0.859773,0.854246,0.864699,0.853668
356,0.852783,0.855900,0.851801,1.000000,0.852571,0.849431,0.853831,0.851902,0.847607,0.846328,...,0.857171,0.848015,0.852222,0.847706,0.851413,0.851289,0.851219,0.852260,0.860721,0.854377
412,0.856118,0.856445,0.860050,0.852571,1.000000,0.855755,0.858976,0.857221,0.854857,0.855991,...,0.859117,0.862847,0.854943,0.850044,0.858385,0.856629,0.855052,0.856608,0.865269,0.862253


Once we had our similarity matrix, then we could produce recommendations for each user and package all the recommendations into a data frame.

In [10]:
recommendations = {}
customers = list(customer_products['CustomerID'].unique())

for customer in customers:
    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])

user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Wine - Redchard Merritt,Bread - Calabrese Baguette,"Thyme - Lemon, Fresh",Milk Powder,Ecolab - Lime - A - Way 4/4 L
1,200,Sauce - Demi Glace,General Purpose Trigger,Cookie Chocolate Chip With,Chef Hat 20cm,Pasta - Angel Hair
2,264,Ezy Change Mophandle,Eggplant - Asian,Scallops - 10/20,Cinnamon Buns Sticky,Wine - Ej Gallo Sierra Valley
3,356,Tea - Herbal Sweet Dreams,Curry Paste - Madras,Tea - English Breakfast,Juice - Orange,Ecolab - Lime - A - Way 4/4 L
4,412,Cake - Box Window 10x10x2.5,Beef - Montreal Smoked Brisket,Bread - Raisin Walnut Oval,"Mushroom - Trumpet, Dry",Cheese - Mix


## Deeper Dive Into Our User Similarity Recommendations

Let's deconstruct what we've done and take a deeper dive into how we put this together. Doing this will equip us with the knowledge to be able to put together an item-based similarity recommender in the next section. 

After creating an empty dictionary to store our recommendations and getting a unique list of customer IDs to iterate through, we are first identifying the top 5 similar customers to the customer we are trying to generate recommendations for. Let's plug in customer ID 33 and see what results we get.

In [11]:
similar_cust = list(cust_dist[33].sort_values(ascending=False)[1:].head().index)
similar_cust

[60862, 27672, 6001, 79458, 33759]

What we get is a list containing the 5 customer IDs of the customers whose purchase behavior is most similar to customer 33. We then go back to our customer_products data frame and select just the purchases where the customer ID is in our list of similar customers. We aggregate on product name, summing up the total quantity purchased of each product by all 5 similar customers, and then we rank them by sorting in descending order by the total quantity.

In [12]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
ranked_products.head()

,ProductName,Quantity
0,Wine - Redchard Merritt,59
1,Cassis,58
2,Bread - Calabrese Baguette,45
3,Wine - Crozes Hermitage E.,45
4,"Thyme - Lemon, Fresh",42


We now have a ranked list of products that similar customers have purchased, but we haven't taken into consideration yet whether our target customer already purchases any of those items. We want to recommend them items that they might like but haven't purchased before. So we will merge the list of ranked products with our target customer's purchase list and keep only the records for items that the customer has not purchased. These will be the items that we recommend to the customer.

In [13]:
merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs

,ProductName,Quantity,Purchased
0,Wine - Redchard Merritt,59,0.0
2,Bread - Calabrese Baguette,45,0.0
4,"Thyme - Lemon, Fresh",42,0.0
9,Milk Powder,39,0.0
10,Ecolab - Lime - A - Way 4/4 L,39,0.0


## Item Similarity Based Recommendations

In this section, you will create an item similarity based recommender system in a step-by-step fashion. Whereas our user similarity based recommender leveraged similarities between customers, this recommender will utilize similarities between products. You already have all the tools in your toolbox, so follow each of the steps below to complete this lab.

### Step 1: Create a product distance matrix.

In [14]:
prod_dist = pd.DataFrame(1/(1 + squareform(pdist(prod_cust_pivot, 'euclidean'))),
                         index=prod_cust_pivot.index, columns=prod_cust_pivot.index)
prod_dist

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,1.000000,0.881127,0.882219,0.884604,0.890449,0.879859,0.885648,0.880852,0.886501,0.890573,...,0.884117,0.888124,0.886567,0.879578,0.875696,0.878427,0.886163,0.886564,0.887956,0.878223
"Appetizer - Mini Egg Roll, Shrimp",0.881127,1.000000,0.883877,0.880604,0.887930,0.881577,0.885241,0.890196,0.888646,0.883484,...,0.887569,0.887831,0.888033,0.881019,0.878720,0.871915,0.888702,0.890837,0.887661,0.882616
Appetizer - Mushroom Tart,0.882219,0.883877,1.000000,0.884708,0.883266,0.884124,0.886495,0.886072,0.887707,0.881019,...,0.886796,0.885968,0.885056,0.887445,0.882789,0.879286,0.890062,0.889960,0.889397,0.881077
Appetizer - Sausage Rolls,0.884604,0.880604,0.884708,1.000000,0.885025,0.885020,0.881992,0.883090,0.880745,0.881226,...,0.882793,0.886507,0.884984,0.886515,0.877195,0.868976,0.885470,0.884688,0.883661,0.877701
Apricots - Dried,0.890449,0.887930,0.883266,0.885025,1.000000,0.886792,0.887542,0.889058,0.886234,0.887162,...,0.886223,0.889001,0.890087,0.887478,0.878571,0.872542,0.892076,0.891209,0.889794,0.886477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.878427,0.871915,0.879286,0.868976,0.872542,0.875465,0.876566,0.877387,0.877911,0.876792,...,0.882272,0.877286,0.874760,0.872730,0.869942,1.000000,0.876589,0.875700,0.877148,0.871735
Yoghurt Tubes,0.886163,0.888702,0.890062,0.885470,0.892076,0.887069,0.893496,0.889616,0.888383,0.888293,...,0.891366,0.892520,0.890222,0.884474,0.885830,0.876589,1.000000,0.889996,0.891179,0.885087
"Yogurt - Blueberry, 175 Gr",0.886564,0.890837,0.889960,0.884688,0.891209,0.891202,0.890707,0.889430,0.891434,0.886262,...,0.890363,0.892818,0.890805,0.885548,0.880785,0.875700,0.889996,1.000000,0.890230,0.886320


### Step 2: Get the products purchased for a specific customer of your choice.

In [15]:
products = list(prod_cust_pivot2[33].loc[prod_cust_pivot2[33]!=0].index)
products[0:10]

['Apricots - Dried',
 'Assorted Desserts',
 'Bandage - Flexible Neon',
 'Bar Mix - Pina Colada, 355 Ml',
 'Beans - Kidney, Canned',
 'Beef - Chuck, Boneless',
 'Beef - Prime Rib Aaa',
 'Beer - Original Organic Lager',
 'Beer - Rickards Red',
 'Black Currants']

### Step 3: For each product the customer purchased, get a list of the top 5 similar products. Package the lists into a nested list, flatten the list, and then filter out any products the customer has already purchased.

In [16]:
similar = {}
for a in products:
    similar[a] = list(prod_dist.loc[a].sort_values(ascending=False)[1:6].index)

In [17]:
lista = [a[1] for a in similar.items()]
listab = [b for paquete in lista for b in paquete ]
listanocomp = [a for a in listab if a not in products]

### Step 4: Count the number of times each similar product occurs in your filtered list. Sort and return a list containing the top 5 items.

Voy a hacer una función para sacar el top five de los productos de la lista de recomendados. Saco una lista de tuplas y hago un set para que ya no se vean los repetidos. Aparece cada producto y las veces que ha aparecido entre los similares de lo que ya ha comprado ese cliente

In [18]:
def topFive(lista):
    return set([(i,lista.count(i)) for i in lista])

In [19]:
topfive = topFive(listanocomp)
ordenada = sorted(topfive, key=lambda tup: (tup[1]),reverse=True)
ordenada[0:5]

[('Sun - Dried Tomatoes', 16),
 ('Spinach - Baby', 14),
 ('Beef - Montreal Smoked Brisket', 11),
 ('Cookies - Assorted', 9),
 ('Rosemary - Primerba, Paste', 9)]

### Step 5: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 2 through 4 for each and appending the results of each iteration to the dictionary you created.

In [20]:
recomend = {}
cust_id = [i for i in data.CustomerID][0:50]

In [21]:
for cust in cust_id:
    products = list(prod_cust_pivot2[cust].loc[prod_cust_pivot2[cust]!=0].index)
    for a in products:
        recomend[cust] = list(prod_dist.loc[a].sort_values(ascending=False)[1:6].index)

### Step 6: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [22]:
recomend_df = pd.DataFrame(recomend)
recomend_df

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,55162,22345,81876,48641,81034,29681,87947,28867,69967,94420
0,Oil - Shortening - All - Purpose,Cheese - Cheddarsliced,Sun - Dried Tomatoes,V8 - Berry Blend,Appetizer - Mushroom Tart,Sun - Dried Tomatoes,Oil - Shortening - All - Purpose,Bread - Italian Roll With Herbs,Sun - Dried Tomatoes,Bread - Italian Roll With Herbs,...,"Cheese - Brie,danish",Oil - Shortening - All - Purpose,Bread - Italian Roll With Herbs,Sun - Dried Tomatoes,Apricots Fresh,Sun - Dried Tomatoes,Scallops - Live In Shell,Oil - Shortening - All - Purpose,Sun - Dried Tomatoes,Sun - Dried Tomatoes
1,"Bread - Roll, Canadian Dinner",Beef - Montreal Smoked Brisket,Oil - Shortening - All - Purpose,Yoghurt Tubes,Turkey - Oven Roast Breast,V8 - Berry Blend,"Bread - Roll, Canadian Dinner","Hickory Smoke, Liquid",V8 - Berry Blend,"Hickory Smoke, Liquid",...,"Chocolate - Semi Sweet, Calets","Bread - Roll, Canadian Dinner","Hickory Smoke, Liquid",V8 - Berry Blend,Beans - Wax,Oil - Shortening - All - Purpose,"Chocolate - Semi Sweet, Calets","Bread - Roll, Canadian Dinner",V8 - Berry Blend,Oil - Shortening - All - Purpose
2,Sun - Dried Tomatoes,Yogurt - French Vanilla,French Pastry - Mini Chocolate,Ocean Spray - Ruby Red,Yogurt - French Vanilla,"Rosemary - Primerba, Paste",Sun - Dried Tomatoes,"Rosemary - Primerba, Paste","Rosemary - Primerba, Paste","Rosemary - Primerba, Paste",...,"Salmon - Atlantic, Skin On",Sun - Dried Tomatoes,"Rosemary - Primerba, Paste","Rosemary - Primerba, Paste","Water - Mineral, Natural",French Pastry - Mini Chocolate,Muffin - Carrot Individual Wrap,Sun - Dried Tomatoes,"Rosemary - Primerba, Paste",French Pastry - Mini Chocolate
3,Bandage - Flexible Neon,Scallops - Live In Shell,Foam Dinner Plate,"Yogurt - Blueberry, 175 Gr","Hickory Smoke, Liquid",Spinach - Baby,Bandage - Flexible Neon,Veal - Osso Bucco,Spinach - Baby,Veal - Osso Bucco,...,"Placemat - Scallop, White",Bandage - Flexible Neon,Veal - Osso Bucco,Spinach - Baby,Ecolab - Mikroklene 4/4 L,Foam Dinner Plate,Veal - Osso Bucco,Bandage - Flexible Neon,Spinach - Baby,Foam Dinner Plate
4,"Rosemary - Primerba, Paste",Napkin White - Starched,Cod - Black Whole Fillet,Muffin - Carrot Individual Wrap,Sun - Dried Tomatoes,Bread - Italian Roll With Herbs,"Rosemary - Primerba, Paste",Rosemary - Dry,Bread - Italian Roll With Herbs,Rosemary - Dry,...,Table Cloth - 53x69 Colour,"Rosemary - Primerba, Paste",Rosemary - Dry,Bread - Italian Roll With Herbs,Beef - Montreal Smoked Brisket,Cod - Black Whole Fillet,"Hickory Smoke, Liquid","Rosemary - Primerba, Paste",Bread - Italian Roll With Herbs,Cod - Black Whole Fillet


## Recommending Items to a New Customer

Suppose we get a new customer and on their first visit, they purchase the following items and quantities.

In [24]:
new_customer = {'Cookies - Assorted':3,
                'Flavouring - Orange':3,
                'Fenngreek Seed':1,
                'Wine - White Cab Sauv.on':1,
                'Bandage - Flexible Neon':3,
                'Oil - Shortening - All - Purpose':2,
                'Beef - Montreal Smoked Brisket':4,
                'French Pastry - Mini Chocolate':4,
                'Snapple Lemon Tea':5,
                'Pepper - White, Ground':2,
                'Spinach - Baby':5,
                'Sole - Dover, Whole, Fresh':4}

### Step 7: Recommend 5 products to this new customer using a user similarity approach.

In [37]:
data2_7 = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'})
data2_7.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

In [38]:
prod7pivot= data2_7.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)

prod7pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
prod7pivot["new"] = pd.Series(new_customer)

In [40]:
prod7pivot = pd.DataFrame.fillna(prod7pivot,value=0)

In [41]:
prod7pivot = prod7pivot.T

In [42]:
prod7pivot.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
dist7 = pd.DataFrame(1/(1 + squareform(pdist(prod7pivot, 'euclidean'))),
                         index=prod7pivot.index, columns=prod7pivot.index)

dist7.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,new
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333,0.067975
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335,0.063707
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333,0.067640
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339,0.063985
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349,0.070847


In [44]:
similarities = dist7['new'].sort_values(ascending=False)[1:]
cust_similar = [i for i in similarities.index]

In [56]:
productosparan = data2_7.loc[cust_similar]

In [57]:
productosparan = productos.groupby(["ProductName"]).agg({"Quantity":"sum"}).sort_values("Quantity",ascending=False)

In [59]:
productosparan

,Quantity
ProductName,
Cookies - Assorted,2403
Flavouring - Orange,2397
Fenngreek Seed,2391
Wine - White Cab Sauv.on,2387
Bandage - Flexible Neon,2347
...,...
Scallops 60/80 Iqf,1546
Rice - Long Grain,1531
Kellogs Special K Cereal,1496


In [58]:
compra_new = [i for i in new_customer.keys()]
recomendadosfinal = [i for i in productosparan.index if i not in compra_new]
recomendadosfinal[0:5]

['Wine - White, Mosel Gold',
 'Tea - Jasmin Green',
 'Kiwi',
 'Cheese - Brie,danish',
 'Soup Knorr Chili With Beans']

### Step 8: Recommend 5 products to this new customer using a item similarity approach.

In [47]:
distancia8 = pd.DataFrame(1/(1 + squareform(pdist(prod_cust_pivot2, 'euclidean'))),
                         index=prod_cust_pivot2.index, columns=prod_cust_pivot2.index)

In [49]:
similar8 = {}
for pr in compra_new:
    similar7[pr] = list(distancia8.loc[pr].sort_values(ascending=False)[1:6].index)

In [50]:
listasim = [z[1] for z in similar7.items()]
listatodo = [x for nestedlist in listasim for x in nestedlist ]
listanocompranew = [c for c in listatodo if c not in compra_new]


In [51]:
top = topFive(listanocompranew)
topsorted = sorted(top, key=lambda f: (f[1]),reverse=True)
topsorted[0:5]

[('Towels - Paper / Kraft', 3),
 ('Rice - Jasmine Sented', 3),
 ('Beef - Chuck, Boneless', 3),
 ('Veal - Osso Bucco', 2),
 ('V8 - Berry Blend', 2)]